In [ ]:
"""

Classification

"""
seed = 42
random_state= seed

import numpy as np
np.random.seed()
import random
random.seed()
import joblib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
import os
import sys
import pickle as pkl
import time
import pandas as pd
# import scipy
# import shap
import math
import sklearn
import scipy.stats as st
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
# from sklearn.svm import SVC
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler,TomekLinks,NeighbourhoodCleaningRule,NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve,auc, classification_report, precision_score, confusion_matrix, recall_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import tree
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
# from scipy import interp
# import category_encoders as ce
# from fancyimpute import KNN
# from fancyimpute import IterativeImputer
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.metrics import mean_squared_error

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
def ElasticNet_function(model_name, hyperparams,X,y,y_flag_positive):
    X = np.array(X)
    y = y
    y_flag_positive = np.array(y_flag_positive)

    # Assign hyper-parameters
    alpha_value, l1_ratio_value = hyperparams
    # Initialize lists with final results
    y_pred_total = []
    y_test_total = []
    # Split data into test and train: random state fixed for reproducibility
    kf = KFold(n_splits=5,shuffle=True,random_state=42)
    # kf-fold cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # y_flag_positive_train, y_flag_positive_test = y_flag_positive[train_index], y_flag_positive[test_index]

        # # only select positive samples for testing
        # positive_index = np.argwhere(y_flag_positive_test == 1).flatten()
        # X_test = X_test[positive_index]
        # y_test = y_test[positive_index]

        # # Scale X_train and X_test
        # scaler = preprocessing.StandardScaler().fit(X_train)
        # X_train_scaled = scaler.transform(X_train)
        # X_test_scaled = scaler.transform(X_test)
        X_train_scaled = X_train
        X_test_scaled = X_test
        # Fit KRR with (X_train_scaled, y_train), and predict X_test_scaled
        if model_name =='elasticNet':
            KRR = ElasticNet(random_state=seed,alpha=alpha_value,l1_ratio=l1_ratio_value) #LinearRegression()
            model_s = KRR.fit(X_train_scaled, y_train)
        # y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        y_pred = model_s.predict(X_test_scaled)
        # Append y_pred and y_test values of this k-fold step to list with total values
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
    # Flatten lists with test and predicted values
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    # Calculate error metric of test and predicted values: rmse
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    # r_pearson,_=pearsonr(y_test_total,y_pred_total)
    # print('KRR k-fold cross-validation . alpha: %7.6f, gamma: %7.4f, RMSE: %7.4f, r: %7.4f' %(alpha_value,gamma_value,rmse,r_pearson))
    print('KRR k-fold cross-validation . alpha: %7.6f, l1_ratio_value: %7.4f, RMSE: %7.4f' % (alpha_value, l1_ratio_value, rmse))
    return rmse, model_s

def create_hyperparams_grid(X,y,y_flag_positive,model_name,params):
    graph_x = []
    graph_y = []
    graph_z = []
    graph_model_s = []
    alpha_value_s = params['alpha']
    l1_ratio_value_s = params['l1_ratio']

    for alpha_value in alpha_value_s:
        # alpha_value = pow(10,alpha_value)
        graph_x_row = []
        graph_y_row = []
        graph_z_row = []
        graph_model_s_row = []
        for l1_ratio_value in l1_ratio_value_s:
            hyperparams = (alpha_value,l1_ratio_value)
            rmse,model_s = ElasticNet_function(model_name, hyperparams, X, y, y_flag_positive)
            graph_x_row.append(alpha_value)
            graph_y_row.append(l1_ratio_value)
            graph_z_row.append(rmse)
            graph_model_s_row.append(model_s)
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)
        graph_model_s.append(graph_model_s_row)
        print('')
    graph_x=np.array(graph_x)
    graph_y=np.array(graph_y)
    graph_z=np.array(graph_z)
    graph_model_s = np.array(graph_model_s)
    min_z = np.min(graph_z)
    pos_min_z = np.argwhere(graph_z == np.min(graph_z))[0]
    print('Minimum RMSE: %.4f' %(min_z))
    print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
    print('Optimum l1_ratio: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))
    best_alpha = graph_x[pos_min_z[0],pos_min_z[1]]
    best_l1_ratio = graph_y[pos_min_z[0],pos_min_z[1]]
    best_model = graph_model_s[pos_min_z[0],pos_min_z[1]]
    best_rmse = min_z

    return best_alpha, best_l1_ratio, best_model, best_rmse

def Build_GridSearch(model_name, params, X_train, y_proba_train, y_train):
    X = X_train.copy()
    y = y_proba_train.copy()
    y_train_flag_positive = y_train.copy()
    best_alpha, best_l1_ratio, best_model, best_rmse = create_hyperparams_grid(X, y, y_train_flag_positive, model_name, params)

    return best_alpha, best_l1_ratio, best_model, best_rmse

def build_classifier_0(X_train,y_train,X_test,y_test,feature_cols,classifier_flag, n_fold,file_path): #X[train], y[train],X[test]
# change label    so as to the objective case is "1", the objective of change label is to keep  classifier.predict_proba(X_text)[:, 1]   "1" is case
    print('---------------------this is classifier_0----------------------')
    print('y_train_number_before_change_label',y_train.value_counts())
    # first replace "0" with 4, then replace 1,2,3, with "0", then replace 4 with "1"
    y_train = y_train.replace(0, 4)
    y_test = y_test.replace(0, 4)

    # then replace 1,2,3, with "0",
    y_train = y_train.replace(1, 0)
    y_train = y_train.replace(2, 0)
    y_train = y_train.replace(3, 0)

    y_test = y_test.replace(1, 0)
    y_test = y_test.replace(2, 0)
    y_test = y_test.replace(3, 0)

    # then replace 4 with "1"
    y_train = y_train.replace(4, 1)
    y_test = y_test.replace(4, 1)

    # print('y_train', y_train)
    print('y_train_number_after_change_label', y_train.value_counts())

# not address imbalance
#     X_train_imbalanced, y_train_imbalanced = X_train, np.array(y_train)

# address imbalance
    cc = SMOTETomek(random_state=42) # ,sampling_strategy={1:num_control}
    # cc = RandomOverSampler(random_state=42)
    # cc = ClusterCentroids(random_state=42)
#
    X_train_imbalanced, y_train_imbalanced = cc.fit_resample(X_train, y_train)


# load the classifier
#     with open(file_path + classifier_flag + '_cv_fold_' + str(n_fold) + '_classifier.pkl', 'rb') as fid:
#         classifier = pkl.load(fid)

# GridSearchCV
    classifier = RandomForestClassifier(random_state=seed)
    params = {'n_estimators': list(range(10, 20, 10)),  # list(range(20,120,10)),
              'max_depth': list(range(2, 17, 2)),
              # 'criterion': ['gini', 'entropy'],
              #  'min_samples_split': list(range(2,10,2)),
              # 'class_weight': ['balanced'],  # , None
              # 'max_leaf_nodes': list(range(2,20,2)),
              # 'max_features': list(range(2,20,2)),
              # 'min_samples_leaf': list(range(2,20,2))
              }
    grid = GridSearchCV(classifier, param_grid=params, scoring='roc_auc', cv=5, n_jobs=4)
    grid.fit(X_train_imbalanced, y_train_imbalanced)
    scores = pd.DataFrame(grid.cv_results_)
    # print('scores',scores)
    grid_accs = scores['mean_test_score'].values.tolist()
    best_auc = grid.best_score_
    best_auc_std = scores.loc[grid.best_index_, 'std_test_score']
    # print(best_auc_std)

    classifier = grid.best_estimator_
#
    print('Best by searching: %s, Std: %s' % (best_auc, best_auc_std))
    best_train_auc= best_auc
    print(grid.best_params_)

# training classifier
#     classifier = RandomForestClassifier(random_state =seed,n_estimators = 100,max_depth =8,class_weight ='balanced')
    classifier.fit(X_train_imbalanced, y_train_imbalanced)

# save the classifier
    with open(file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_classifier.pkl', 'wb') as fid:
        pkl.dump(classifier, fid)

    y_proba = classifier.predict_proba(X_test)[:, 1]  #
    # y_predic = classifier.predict(X_test)
    # print('y_predic',y_predic)

    test_result = y_proba
    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)

#  plot feature importance
    X_data = X_train_imbalanced
    feature_cols = feature_cols

    # fig, ax = plt.subplots(figsize=(15, 12))
    # fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)
    #
    importances = classifier.feature_importances_
    # # print('importances',importances)
    num_features = len(importances)
    # indices = np.argsort(abs(importances))[::-1]
    # plt.title("Feature importance")
    # plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    # plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])
    #
    # # plt.show()
    # fig_name = file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.png'
    # fig.savefig(fig_name)
    # plt.close('all')

    # print(importances.shape)
    importances_df = pd.DataFrame(importances.reshape((1,-1)),columns=feature_cols)
    importances_df.to_csv(file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.csv',index=False,header=True)

    stu_model = DecisionTreeRegressor(max_depth=5)
    y_proba_train = classifier.predict_proba(X_train)[:, 1]
    stu_model.fit(X_train, y_proba_train)

    y_proba_test_stu = stu_model.predict(X_test)

    return test_result

def build_classifier_1(X_train,y_train,X_test,y_test,feature_cols,classifier_flag, n_fold,file_path): #X[train], y[train],X[test]
# change label    so as to the objective case is "1", the objective of change label is to keep  classifier.predict_proba(X_text)[:, 1]   "1" is case
    print('---------------------this is classifier_1----------------------')
    print('y_train_number_before_change_label',y_train.value_counts())
    # replace 2,3 with 0,
    y_train = y_train.replace(2, 0)
    y_train = y_train.replace(3, 0)
    y_test = y_test.replace(2, 0)
    y_test = y_test.replace(3, 0)

    # print('y_train', y_train)
    print('y_train_number_after_change_label', y_train.value_counts())

# GridSearchCV
    classifier = RandomForestClassifier(random_state=seed)
    # classifier = DecisionTreeClassifier(random_state=seed)
    # classifier = LogisticRegression(random_state=seed,max_iter = 200)
    #
    # params = {
    #         'C': [0.7],# 0.08, 0.1, 0.3, 0.5, 0.6, 1, 2, 3, 4,
    #         # 'l1_ratio':[0.3],
    #         'penalty':['l1'], # 'elasticnet'
    #         'solver':['saga'] # liblinear
    #         }

    params = {
              'n_estimators': [100], # for 24hours, worst, [110]
              'max_depth': [6], # for 24hours, mean, [10],for 24hours, worst, [12]
              # 'criterion': ['gini', 'entropy'],
              #  'min_samples_split': list(range(2,10,2)),
              'class_weight': ['balanced'],  # , None
              # 'max_leaf_nodes': list(range(2,10,2)),
              # 'max_features': list(range(2,20,2)),
              # 'min_samples_leaf': list(range(2,20,2))
              }

    grid = GridSearchCV(classifier, param_grid=params, scoring='roc_auc', cv=5, n_jobs=4) # scoring='roc_auc', 'precision'
    grid.fit(X_train, y_train)
    scores = pd.DataFrame(grid.cv_results_)
    # print('scores',scores)
    grid_accs = scores['mean_test_score'].values.tolist()
    best_acc = grid.best_score_
    best_acc_std = scores.loc[grid.best_index_, 'std_test_score']
    # print(best_auc_std)
    classifier = grid.best_estimator_
    print('Best by searching: %s, Std: %s' % (best_acc, best_acc_std))
    print(grid.best_params_)

# training classifier on all train data
    classifier.fit(X_train, y_train)
    y_proba = classifier.predict_proba(X_test)[:, 1]
    y_pred = classifier.predict(X_test)
    test_result = y_proba

    # print('y_proba',y_proba)
    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    # print('precision',precision)
    # print('recall',recall)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)

    precision_RI = precision_score(y_test, y_pred)
    recall_RI = recall_score(y_test, y_pred)
    confusion_matrix_RI = confusion_matrix(y_test, y_pred)

    print('precision_RI:', precision_RI)
    print('recall_RI:', recall_RI)
    print('confusion_matrix_RI:', confusion_matrix_RI)

    # # plot precision recall curve
    # fig, ax = plt.subplots()
    # ax.plot(recall, precision, color='purple')
    #
    # #add axis labels to plot
    # ax.set_title('RI-Precision-Recall Curve')
    # ax.set_ylabel('Precision')
    # ax.set_xlabel('Recall')
    # plt.ylim(bottom=0)
    # #display plot
    # # plt.show()
    # current_time = datetime.now()
    # fig_name_0 = "RI_Precision_Recall_curve"+"_"+str(current_time)+".png"
    # fig.savefig(file_path + fig_name_0)
    # plt.close('all')

# plot decision tree
#     fig = plt.figure(figsize=(50, 30))
#     _ = tree.plot_tree(classifier,
#                        feature_names=feature_cols,
#                        class_names=["Not Ripidly Improving", "Ripidly Improving"],
#                        filled=True, fontsize=20)
#
#     fig_name = "RI_visualization.png"
#     fig.savefig(file_path + fig_name)
#     plt.close('all')

# save the classifier
    with open(file_path + 'RI_ori.pkl', 'wb') as fid:
        pkl.dump(classifier, fid)

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    importances = classifier.feature_importances_
    # importances = classifier.coef_.reshape(-1)

    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1] #
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # plt.show()
    fig_name = file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.png'
    fig.savefig(fig_name)
    plt.close('all')

    # print(importances.shape)
    importances_df = pd.DataFrame(importances.reshape((1,-1)),columns=feature_cols)
    importances_df.to_csv(file_path+classifier_flag+'_teacher'+'.csv',index=False,header=True)

    if num_sel_feat > 0:
        sel_idx = np.argsort(importances)[::-1][:num_sel_feat]
        X_train = X_train.iloc[:, sel_idx]
        X_test = X_test.iloc[:, sel_idx]

    if distill_method == "mimic_teacher":
        if distill_model == "tree":
            stu_model = DecisionTreeRegressor(max_depth=4)
        elif distill_model == "lr":
            stu_model = ElasticNet(random_state=seed,normalize=True) #LinearRegression()
            # stu_model = LogisticRegression()
            # tune
            params = {'alpha':[0.00003], # 0.0003, 0.00025,0.0004, 0.00005,0.00045, 0.01
                    'l1_ratio': [1],
                    }
        else:
            raise NotImplementedError

        y_proba_train = classifier.predict_proba(X_train)[:, 1]
        y_proba_test = classifier.predict_proba(X_test)[:, 1]
        y_pos_proba_test = classifier.predict_proba(X_test.loc[y_test[y_test == 1].index])[:, 1]

        grid = GridSearchCV(stu_model, param_grid=params, scoring='neg_mean_squared_error',  cv=5, n_jobs=4)  # scoring='roc_auc', 'precision'
        grid.fit(X_train, y_proba_train)
        stu_model = grid.best_estimator_
        print('Best by searching: %s, Std: %s' % (best_acc, best_acc_std))
        print(grid.best_params_)

        # # build GridSearch
        # model_name = 'elasticNet'
        # # only select positive samples
        # positive_index = list(y_train[y_train == 1].index)
        # X_train = X_train.loc[positive_index]
        # positive_index_in_proba = np.argwhere(np.array(y_train) == 1).flatten()
        # y_proba_train = y_proba_train[positive_index_in_proba]
        #
        # best_alpha,best_l1_ratio,best_model,best_rmse = Build_GridSearch(model_name, params, X_train, y_proba_train, y_train) # y_train for selecting only positive samples
        # print('best model hyperparameters:',best_alpha, best_l1_ratio)
        # print('best model hyperparameters----best_rmse:', best_rmse)
        # stu_model = best_model

        # training classifier on all train data
        stu_model.fit(X_train, y_proba_train)
        print('stu_model.coef_',stu_model.coef_)

        # save the stu_model
        with open(file_path + 'RI_simplified.pkl', 'wb') as fid:
            pkl.dump(stu_model, fid)

        # with open(file_path+'RI_simplified.pkl', 'rb') as file:
        #     dataee = pickle.load(file)
        # print(dataee)

    elif distill_method == "train_from_scratch":
        if distill_model == "tree":
            stu_model = DecisionTreeClassifier(max_depth=4)
        elif distill_model == "lr":
            stu_model = LogisticRegression()
        else:
            raise NotImplementedError
        stu_model.fit(X_train, y_train)
    else:
        raise NotImplementedError

    print("For Student Model:")
    y_proba_test_stu = stu_model.predict(X_test) #

    stu_model_rmse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=False)
    stu_model_mse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=True)
    print('stu_model_rmse',stu_model_rmse)
    print('stu_model_mse',stu_model_mse)

    # only focus on positive sample's rmse and mse
    X_pos_test = X_test.loc[y_test[y_test == 1].index]
    y_pos_proba_test_stu = stu_model.predict(X_pos_test)
    stu_pos_model_rmse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=False)
    stu_pos_model_mse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=True)
    print('stu_pos_model_rmse', stu_pos_model_rmse)
    print('stu_pos_model_mse', stu_pos_model_mse)

    # compute true positive
    RI_threshold = 0.5 # 0.65
    rec_pos = np.count_nonzero(y_pos_proba_test_stu >= RI_threshold)/len(y_pos_proba_test_stu)
    print('rec_pos',rec_pos)
    y_pred_test_stu = y_proba_test_stu.copy()
    y_pred_test_stu[y_pred_test_stu>=RI_threshold] = 1
    y_pred_test_stu[y_pred_test_stu<RI_threshold] = 0

    precision_RI_stu = precision_score(y_test, y_pred_test_stu)
    print('precision_RI_stu:', precision_RI_stu)
    recall_RI_stu = recall_score(y_test, y_pred_test_stu)
    print('recall_RI_stu:', recall_RI_stu)
    confusion_matrix_RI_stu = confusion_matrix(y_test, y_pred_test_stu)
    print('confusion_matrix_stu:', confusion_matrix_RI_stu)

    aucroc = roc_auc_score(y_test, y_proba_test_stu)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, _ = precision_recall_curve(y_test, y_proba_test_stu)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)


    # # # plot precision recall curve
    # fig, ax = plt.subplots()
    # ax.plot(recall, precision, color='purple')
    #
    # #add axis labels to plot
    # ax.set_title('stu_RI-Precision-Recall Curve')
    # ax.set_ylabel('Precision')
    # ax.set_xlabel('Recall')
    # plt.ylim(bottom=0)
    # #display plot
    # # plt.show()
    # current_time = datetime.now()
    # fig_name_0 = "stu_RI_Precision_Recall_curve"+"_"+str(current_time)+".png"
    # fig.savefig(file_path + fig_name_0)
    # plt.close('all')

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    importances = stu_model.coef_
    # importances = stu_model.coef_.reshape(-1)

    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1] #
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # plt.show()
    fig_name = file_path+classifier_flag+'_student_'+'.png'
    fig.savefig(fig_name)
    plt.close('all')

    return test_result
    # return y_proba_test_stu


def build_classifier_3(X_train,y_train,X_test,y_test,feature_cols,classifier_flag, n_fold,file_path): #X[train], y[train],X[test]
# change label    so as to the objective case is "1", the objective of change label is to keep  classifier.predict_proba(X_text)[:, 1]   "1" is case
    print('---------------------this is classifier_3----------------------')
    print('y_train_number_before_change_label',y_train.value_counts())

    # first replace 1,2 with 0, then replace 3, with 1
    y_train = y_train.replace(1, 0)
    y_train = y_train.replace(2, 0)
    y_test = y_test.replace(1, 0)
    y_test = y_test.replace(2, 0)

    # then replace 3 with 1,
    y_train = y_train.replace(3, 1)
    y_test = y_test.replace(3, 1)

    # print('y_train', y_train)
    print('y_train_number_after_change_label', y_train.value_counts())

# not address imbalance
#     X_train_imbalanced, y_train_imbalanced = X_train, y_train

    # address imbalance
    # cc = SMOTETomek(random_state=42,sampling_strategy='minority')
    # cc = RandomOverSampler(random_state=42)
    # cc = ClusterCentroids(random_state=42)

    # X_train, y_train = cc.fit_resample(X_train, y_train)

# GridSearchCV
    classifier = RandomForestClassifier(random_state=seed)
#     classifier = DecisionTreeClassifier(random_state=seed)
    params = {
               'n_estimators': [110],  # list(range(20,120,10)), for 6hours: [100], for 24hours, mean, [110]; for 24hours, worst, [110]
              'max_depth': [4], # for 6hour: [8], for 24hours, mean, [8] # list(range(2,20,2)) # for 24 hours, worst, [6],[10]
              # 'criterion': ['gini', 'entropy'], # 'gini', 'entropy'
              #  'min_samples_split': [8], # list(range(2,10,2))
              'class_weight': ['balanced'],  # , None # for 24 hours, worst, [balanced]
              # 'max_leaf_nodes': list(range(2,20,2)),
              # 'max_features': list(range(2,20,2)),
              # 'min_samples_leaf': list(range(2,20,2))
              }
    grid = GridSearchCV(classifier, param_grid=params, scoring='roc_auc', cv=5, n_jobs=4) # scoring='roc_auc'
    grid.fit(X_train, y_train)
    scores = pd.DataFrame(grid.cv_results_)
    # print('scores',scores)
    grid_accs = scores['mean_test_score'].values.tolist()
    best_auc = grid.best_score_
    best_auc_std = scores.loc[grid.best_index_, 'std_test_score']
    # print(best_auc_std)
    classifier = grid.best_estimator_

    print('Best by searching: %s, Std: %s' % (best_auc, best_auc_std))
    best_train_auc= best_auc
    print(grid.best_params_)

    # load the classifier
    # with open(file_path + 'ori_RW.pkl', 'rb') as fid:
    #     classifier = pkl.load(fid)

# training classifier
    classifier.fit(X_train, y_train)

    y_proba = classifier.predict_proba(X_test)[:, 1]
    y_pred = classifier.predict(X_test)
    test_result = y_proba
    # print('y_proba',y_proba)
    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    # print('precision',precision)
    # print('recall',recall)
    # print('thresholds',thresholds)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)

    precision_RW = precision_score(y_test, y_pred)
    recall_RW = recall_score(y_test, y_pred)
    confusion_matrix_RW = confusion_matrix(y_test, y_pred)

    print('precision_RW:', precision_RW)
    print('recall_RW:', recall_RW)
    print('confusion_matrix:', confusion_matrix_RW)

# # plot precision recall curve
    # fig, ax = plt.subplots()
    # ax.plot(recall, precision, color='purple')
    #
    # # add axis labels to plot
    # ax.set_title('RW-Precision-Recall Curve')
    # ax.set_ylabel('Precision')
    # ax.set_xlabel('Recall')
    # plt.ylim(bottom=0)
    # # display plot
    # # plt.show()
    # current_time = datetime.now()
    # fig_name_0 = "RW_Precision_Recall_curve"+"_"+str(current_time)+".png"
    # fig.savefig(file_path + fig_name_0)
    # plt.close('all')

    # save the classifier
    with open(file_path + 'RW_ori.pkl', 'wb') as fid:
        pkl.dump(classifier, fid)

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    importances = classifier.feature_importances_
    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1] #
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # plt.show()
    fig_name = file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.png'
    fig.savefig(fig_name)
    plt.close('all')
#
#     # print(importances.shape)
#     importances_df = pd.DataFrame(importances.reshape((1,-1)),columns=feature_cols)
#     importances_df.to_csv(file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.csv',index=False,header=True)
#
    if distill_method == "mimic_teacher":
        if distill_model == "tree":
            stu_model = DecisionTreeRegressor(max_depth=4)
        elif distill_model == "lr":
            # stu_model = LinearRegression()
            stu_model = ElasticNet(random_state=seed,normalize=True) #LinearRegression()
            # stu_model = LogisticRegression()
            # tune
            params = {'alpha':[0.00003], #[0.0003,0.1,0.01,0.0001,1,2] # 0.000003, 0.0003, 0.00025
                'l1_ratio': [1],
                    }
        else:
            raise NotImplementedError

        y_proba_train = classifier.predict_proba(X_train)[:, 1]
        y_proba_test = classifier.predict_proba(X_test)[:, 1]
        y_pos_proba_test = classifier.predict_proba(X_test.loc[y_test[y_test == 1].index])[:, 1]

        grid = GridSearchCV(stu_model, param_grid=params, scoring='neg_mean_squared_error',  cv=5, n_jobs=4)  # scoring='roc_auc', 'precision'
        grid.fit(X_train, y_proba_train)
        stu_model = grid.best_estimator_
        # print('Best by searching: %s, Std: %s' % (best_acc, best_acc_std))
        print(grid.best_params_)

        # # only select positive samples for testing
        # positive_index = list(y_train[y_train == 1].index)
        # X_train = X_train.loc[positive_index]
        # positive_index_in_proba = np.argwhere(np.array(y_train) == 1).flatten()
        # y_proba_train = y_proba_train[positive_index_in_proba]
        # # build GridSearch
        # model_name = 'elasticNet'
        # best_alpha,best_l1_ratio,best_model,best_rmse = Build_GridSearch(model_name, params, X_train, y_proba_train, y_train) # y_train for selecting only positive samples
        # # print('best model hyperparameters:',best_alpha, best_l1_ratio)
        # # print('best model hyperparameters----best_rmse:', best_rmse)
        # stu_model = best_model

        # training classifier on all train data
        stu_model.fit(X_train, y_proba_train)
        print('stu_model.coef_',stu_model.coef_)

        # save the stu_model
        with open(file_path + 'RW_simplified.pkl', 'wb') as fid:
            pkl.dump(stu_model, fid)

    elif distill_method == "train_from_scratch":
        if distill_model == "tree":
            stu_model = DecisionTreeClassifier(max_depth=4)
        elif distill_model == "lr":
            stu_model = LogisticRegression()
        else:
            raise NotImplementedError
        stu_model.fit(X_train, y_train)
    else:
        raise NotImplementedError

    print("For Student Model:")
    y_proba_test_stu = stu_model.predict(X_test)

    stu_model_rmse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=False)
    stu_model_mse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=True)
    print('stu_model_rmse', stu_model_rmse)
    print('stu_model_mse', stu_model_mse)

    # only focus on positive sample's rmse and mse
    X_pos_test = X_test.loc[y_test[y_test == 1].index]
    y_pos_proba_test_stu = stu_model.predict(X_pos_test)
    stu_pos_model_rmse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=False)
    stu_pos_model_mse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=True)
    print('stu_pos_model_rmse', stu_pos_model_rmse)
    print('stu_pos_model_mse', stu_pos_model_mse)

# compute true positive
    RW_threshold = 0.50  # 0.55
    pre_pos = np.count_nonzero(y_pos_proba_test_stu >= 0.5) / len(y_pos_proba_test_stu)
    # print('pre_pos', pre_pos)
    y_pred_test_stu = y_proba_test_stu.copy()
    y_pred_test_stu[y_pred_test_stu >= RW_threshold] = 1
    y_pred_test_stu[y_pred_test_stu < RW_threshold] = 0

    precision_RW_stu = precision_score(y_test, y_pred_test_stu)
    print('precision_RW_stu:', precision_RW_stu)
    recall_RW_stu = recall_score(y_test, y_pred_test_stu)
    print('recall_RW_stu:', recall_RW_stu)
    confusion_matrix_RW_stu = confusion_matrix(y_test, y_pred_test_stu)
    print('confusion_matrix_stu:', confusion_matrix_RW_stu)

    aucroc = roc_auc_score(y_test, y_proba_test_stu)
    print('aucroc', aucroc)
    # # precision-recall auc
    precision, recall, _ = precision_recall_curve(y_test, y_proba_test_stu)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)

    # # plot precision recall curve
    # fig, ax = plt.subplots()
    # ax.plot(recall, precision, color='purple')
    #
    # # add axis labels to plot
    # ax.set_title('stu_RW-Precision-Recall Curve')
    # ax.set_ylabel('Precision')
    # ax.set_xlabel('Recall')
    # plt.ylim(bottom=0)
    # # display plot
    # # plt.show()
    # current_time = datetime.now()
    # fig_name_0 = "stu_RW_Precision_Recall_curve" + "_" + str(current_time) + ".png"
    # fig.savefig(file_path + fig_name_0)
    # plt.close('all')

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    importances = stu_model.coef_
    # importances = stu_model.coef_.reshape(-1)

    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1]
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # plt.show()
    fig_name = file_path+classifier_flag+'_student_'+'.png'
    fig.savefig(fig_name)
    plt.close('all')

    return test_result
    # return y_proba_test_stu


def compute_xAUC_RI(C_1_result, C_3_result, y_test, file_path):
    RI_proba = []
    RI_y_test = []
    y_test = list(y_test)
    for i in range(len(C_1_result)):
        C_y_test = y_test[i]
        if C_y_test != 3:  # keeping RI group
            # if C_y_test != 1:  # keeping RW group
            if C_y_test == 1:  # for RI group
                # if C_y_test == 3: # for RW group
                RI_y_test.append(1)
                C_proba = C_1_result[i]
                RI_proba.append(C_proba)
            else:
                RI_y_test.append(0)
                C_proba = C_3_result[i]
                RI_proba.append(C_proba)

    y_proba = np.array(RI_proba)
    y_test = np.array(RI_y_test)

    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    # print('precision', precision)
    # print('recall', recall)
    aucpr = auc(recall, precision)

    print('aucpr', aucpr)
    # # plot precision recall curve
    fig, ax = plt.subplots()
    ax.plot(recall, precision, color='purple')

    # add axis labels to plot
    ax.set_title('RI-Precision-Recall Curve')
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')
    plt.ylim(bottom=0)
    # display plot
    # plt.show()
    current_time = datetime.now()
    fig_name_0 = "xAUC" + "_RI_" + str(current_time) + ".png"
    fig.savefig(file_path + fig_name_0)
    plt.close('all')

    return aucroc, aucpr

def compute_xAUC_RW(C_1_result, C_3_result, y_test, file_path):
    RI_proba = []
    RI_y_test = []
    y_test = list(y_test)
    for i in range(len(C_1_result)):
        C_y_test = y_test[i]
        # if C_y_test != 3:  # keeping RI group
        if C_y_test != 1:  # keeping RW group
            # if C_y_test == 1:  # for RI group
            if C_y_test == 3:  # for RW group
                RI_y_test.append(1)
                C_proba = C_1_result[i]
                RI_proba.append(C_proba)
            else:
                RI_y_test.append(0)
                C_proba = C_3_result[i]
                RI_proba.append(C_proba)

    y_proba = np.array(RI_proba)
    y_test = np.array(RI_y_test)

    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    # print('precision', precision)
    # print('recall', recall)
    aucpr = auc(recall, precision)

    print('aucpr', aucpr)
    # # plot precision recall curve
    fig, ax = plt.subplots()
    ax.plot(recall, precision, color='purple')

    # add axis labels to plot
    ax.set_title('RW-Precision-Recall Curve')
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')
    plt.ylim(bottom=0)
    # display plot
    # plt.show()
    current_time = datetime.now()
    fig_name_0 = "xAUC" + "_RW_" + str(current_time) + ".png"
    fig.savefig(file_path + fig_name_0)
    plt.close('all')
    return aucroc, aucpr


def change_label(data_df, case_flag):
    data_df = data_df

    if case_flag == 0:
        # first change case label 0->4;
        data_df.loc[data_df['group'] == 0, 'group'] = 4

        # then change control lable 1->0, 2->0, 3->0
        data_df.loc[data_df['group'] == 1, 'group'] = 0
        data_df.loc[data_df['group'] == 2, 'group'] = 0
        data_df.loc[data_df['group'] == 3, 'group'] = 0

        # third change case label 4->1;  "1" is fouced.
        data_df.loc[data_df['group'] == 4, 'group'] = 1

    if case_flag == 1:
        # change label, 0->0, 2->0, 3->0; ------1 vs rest
        data_df.loc[data_df['group'] == 0, 'group'] = 0
        data_df.loc[data_df['group'] == 2, 'group'] = 0
        data_df.loc[data_df['group'] == 3, 'group'] = 0
    if case_flag == 2:
        # first change control lable 0->0, 1->0, 3->0
        data_df.loc[data_df['group'] == 0, 'group'] = 0
        data_df.loc[data_df['group'] == 1, 'group'] = 0
        data_df.loc[data_df['group'] == 3, 'group'] = 0

        # then change case label 2->1;  "1" is fouced.
        data_df.loc[data_df['group'] == 2, 'group'] = 1

    if case_flag == 3:
        # first change control lable 0->0, 1->0, 2->0
        data_df.loc[data_df['group'] == 0, 'group'] = 0
        data_df.loc[data_df['group'] == 1, 'group'] = 0
        data_df.loc[data_df['group'] == 2, 'group'] = 0

        # then change case label 3->1;  "1" is fouced.
        data_df.loc[data_df['group'] == 3, 'group'] = 1

    return data_df

def clean_outliers(df,cols):
    # cols: list
    for col in cols:
        min_value = df[col].quantile(0.01)
        max_value = df[col].quantile(0.99)
        df[col][df[col] < min_value] = np.nan
        df[col][df[col] > max_value] = np.nan
    return df

In [ ]:
file_path_data_mimic = 'Subtyping/MIMIC-IV/data/'
file_path_data_eicu = 'Subtyping/eICU/data/'
file_path = 'Subtyping/models/'

dataset_flag = 'combined'# MIMIC, eICU, 'combined','M_train_e_test'# train on MIMIC, test on
two_set_flag = 0

num_sel_feat = -1 # -1 to keep all the features
# "train_from_scratch" or "mimic_teacher"
distill_method = "train_from_scratch" # "train_from_scratch"
# "tree" or "lr"
distill_model = "lr"

start_time = time.time()

prediction_feature = 'prediction_feature_6'

# read data
data_df_ori_1 = pd.read_csv(file_path_data_mimic + 'features_24h.csv',dtype={'patientunitstayid':str}, index_col=0) # Note that, using patientunitstayid as index, MIMIC: data_df_ori  24 hours data

data_df_ori_2 = pd.read_csv(file_path_data_eicu + 'features_24h.csv', dtype={'patientunitstayid':str}, index_col=0) # Note that, using patientunitstayid as index, eICU: data_df_ori, 24 hours

print('len(data_df_ori_1)',len(data_df_ori_1))
print('len(data_df_ori_2)',len(data_df_ori_2))


In [ ]:

# remove outlier values
outlier_cols = ['Bands', 'CRP', 'Temperature', 'WBC', 'SO2', 'Respiratory_rate',
             'Heart_rate', 'Lactate', 'Systolic_ABP', 'BUN', 'Creatinine',
            'ALT', 'AST', 'Bilirubin','GCS','Hemoglobin', 'INR', 'Platelet','Albumin',
            'Chloride', 'Glucose', 'Sodium', 'BMI'] # no 'Age', 'Comorbidity_score', 'Pao2',

data_df_ori_1 = clean_outliers(data_df_ori_1, outlier_cols)
data_df_ori_2 = clean_outliers(data_df_ori_2, outlier_cols)

data_df_ori_1.replace([np.inf, -np.inf], np.nan, inplace=True)
data_df_ori_2.replace([np.inf, -np.inf], np.nan, inplace=True)

data_df_1 = data_df_ori_1.fillna(data_df_ori_1.median())
data_df_2 = data_df_ori_2.fillna(data_df_ori_2.median())


In [ ]:

drug_variable = ['Dopamine_total', 'Dobutamine_total', 'Norepinephrine_total', 'Epinephrine_total', 'Phenylephrine_total', 'Vasopressin_total','MAP', 'MAP_NEQ','Total_DDNEP']
# drug_variable = ['MAP_NEQ','Total_DDNEP']

obj_feature_name = ['SOFA_score', 'Respiration_score', 'Coagulation_score', 'Liver_score',
                     'Cardiovascular_score','CNS_score', 'Renal_score',
             'CRP', 'Temperature', 'WBC', 'SO2', 'Pao2', 'Respiratory_rate',
            'Heart_rate', 'Lactate', 'Systolic_ABP','BUN', 'Creatinine',
            'ALT', 'AST', 'Bilirubin','GCS','Hemoglobin', 'INR', 'Platelet',
            'Chloride', 'Glucose', 'Sodium', 'BMI',
            'Age']


if dataset_flag =='eICU':
    X_train, X_test, y_train, y_test = train_test_split(data_df_2[obj_feature_name], data_df_2['group'], test_size=0.2, random_state=seed)

elif dataset_flag =='MIMIC':
    X_train, X_test, y_train, y_test = train_test_split(data_df_1[obj_feature_name], data_df_1['group'], test_size=0.2, random_state=seed)

elif dataset_flag == 'M_train_e_test': # train on MIMIC and testing on eICU
    X_train = data_df_1[obj_feature_name]
    X_test = data_df_2[obj_feature_name]
    y_train = data_df_1['group']
    y_test = data_df_2['group']
else:
    two_set_flag = 1
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(data_df_1[obj_feature_name], data_df_1['group'], test_size=0.2, random_state=seed)
    X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(data_df_2[obj_feature_name], data_df_2['group'], test_size=0.2, random_state=seed)

    X_train = pd.concat([X_train_1, X_train_2], axis=0, ignore_index=True) # , ignore_index=True
    y_train = pd.concat([y_train_1, y_train_2], axis=0, ignore_index=True) # , ignore_index=True

    X_test = pd.concat([X_test_1, X_test_2], axis=0, ignore_index=True) # , ignore_index=True
    y_test = pd.concat([y_test_1, y_test_2], axis=0, ignore_index=True) # , ignore_index=True

X_ori = pd.concat([X_train, X_test], axis=0, ignore_index=True)
y_ori = pd.concat([y_train, y_test], axis=0, ignore_index=True)
print('len X:', len(X_train))
print('len y:', len(y_train))

feature_cols = obj_feature_name
n_fold = 0


In [ ]:
def build_classifier(X_train,y_train,X_test,y_test,feature_cols,classifier_flag, n_fold,file_path,group): #X[train], y[train],X[test]
# change label    so as to the objective case is "1", the objective of change label is to keep  classifier.predict_proba(X_text)[:, 1]   "1" is case
    print('---------------------this is classifier_1----------------------')
    print('y_train_number_before_change_label',y_train.value_counts())
    # replace 2,3 with 0,
    if group == 1:
        save_label = 'RI'
        y_train = y_train.replace(2, 0)
        y_train = y_train.replace(3, 0)
        y_test = y_test.replace(2, 0)
        y_test = y_test.replace(3, 0)
    if group == 3:
        save_label = 'RW'
        y_train = y_train.replace(1, 0)
        y_train = y_train.replace(2, 0)
        y_test = y_test.replace(1, 0)
        y_test = y_test.replace(2, 0)

        y_train = y_train.replace(3, 1)
        y_test = y_test.replace(3, 1)


    # print('y_train', y_train)
    print('y_train_number_after_change_label', y_train.value_counts())

# GridSearchCV
    classifier = RandomForestClassifier(random_state=seed)

    params = {
              'n_estimators': [110], # for 24hours, worst, [110]
              'max_depth': [10], # for 24hours, mean, [10],for 24hours, worst, [12]
              'criterion': ['gini'],
               'min_samples_split': [2],
              'class_weight': ['balanced'],  # , None
              'max_leaf_nodes': [8],
              'max_features': [2],
              'min_samples_leaf': [12]
              }

    grid = GridSearchCV(classifier, param_grid=params, scoring='roc_auc', cv=5, n_jobs=4) # scoring='roc_auc', 'precision'
    grid.fit(X_train, y_train)
    scores = pd.DataFrame(grid.cv_results_)
    # print('scores',scores)
    grid_accs = scores['mean_test_score'].values.tolist()
    best_acc = grid.best_score_
    best_acc_std = scores.loc[grid.best_index_, 'std_test_score']
    # print(best_auc_std)
    classifier = grid.best_estimator_
    print('Best by searching: %s, Std: %s' % (best_acc, best_acc_std))
    print(grid.best_params_)

# training classifier on all train data
    classifier.fit(X_train, y_train)
    y_proba = classifier.predict_proba(X_test)[:, 1]
    y_pred = classifier.predict(X_test)
    test_result = y_proba

    aucroc = roc_auc_score(y_test, y_proba)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)

    precision_RI = precision_score(y_test, y_pred)
    recall_RI = recall_score(y_test, y_pred)
    confusion_matrix_RI = confusion_matrix(y_test, y_pred)

    print('precision_RI:', precision_RI)
    print('recall_RI:', recall_RI)
    print('confusion_matrix_RI:', confusion_matrix_RI)

    # # plot precision recall curve
    # fig, ax = plt.subplots()
    # ax.plot(recall, precision, color='purple')
    #
    # #add axis labels to plot
    # ax.set_title('RI-Precision-Recall Curve')
    # ax.set_ylabel('Precision')
    # ax.set_xlabel('Recall')
    # plt.ylim(bottom=0)
    # #display plot
    # # plt.show()
    # current_time = datetime.now()
    # fig_name_0 = "RI_Precision_Recall_curve"+"_"+str(current_time)+".png"
    # fig.savefig(file_path + fig_name_0)
    # plt.close('all')

# plot decision tree
#     fig = plt.figure(figsize=(50, 30))
#     _ = tree.plot_tree(classifier,
#                        feature_names=feature_cols,
#                        class_names=["Not Ripidly Improving", "Ripidly Improving"],
#                        filled=True, fontsize=20)
#
#     fig_name = "RI_visualization.png"
#     fig.savefig(file_path + fig_name)
#     plt.close('all')

# # save the classifier
#     with open(file_path + f'{save_label}_ori.pkl', 'wb') as fid:
#         pkl.dump(classifier, fid)

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    importances = classifier.feature_importances_
    # importances = classifier.coef_.reshape(-1)

    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1] #
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # plt.show()
    fig_name = file_path+classifier_flag+'_cv_fold_'+str(n_fold)+'_feature_importance_aucroc'+str(aucroc)+'_aucpr_'+str(aucpr)+'.png'
    fig.savefig(fig_name, facecolor='white', transparent=False)
    plt.close('all')

    # print(importances.shape)
    importances_df = pd.DataFrame(importances.reshape((1,-1)),columns=feature_cols)
    importances_df.to_csv(file_path+classifier_flag+'_teacher'+'.csv',index=False,header=True)

    if num_sel_feat > 0:
        sel_idx = np.argsort(importances)[::-1][:num_sel_feat]
        X_train = X_train.iloc[:, sel_idx]
        X_test = X_test.iloc[:, sel_idx]

    y_proba_train = classifier.predict_proba(X_train)[:, 1]
    y_proba_test = classifier.predict_proba(X_test)[:, 1]
    y_pos_proba_test = classifier.predict_proba(X_test.loc[y_test[y_test == 1].index])[:, 1]

    if distill_method == "mimic_teacher":
        if distill_model == "tree":
            stu_model = DecisionTreeRegressor(max_depth=4)
        elif distill_model == "lr":
            stu_model = ElasticNet(random_state=seed,normalize=True) #LinearRegression()
            # stu_model = LogisticRegression()
            # tune
            params = {'alpha':[0.00003], # 0.0003, 0.00025,0.0004, 0.00005,0.00045, 0.01
                    'l1_ratio': [1],
                    }
        else:
            raise NotImplementedError

        grid = GridSearchCV(stu_model, param_grid=params, scoring='neg_mean_squared_error',  cv=5, n_jobs=4)  # scoring='roc_auc', 'precision'
        grid.fit(X_train, y_proba_train)
        stu_model = grid.best_estimator_
        print('Best by searching: %s, Std: %s' % (best_acc, best_acc_std))
        print(grid.best_params_)

        # # build GridSearch
        # model_name = 'elasticNet'
        # # only select positive samples
        # positive_index = list(y_train[y_train == 1].index)
        # X_train = X_train.loc[positive_index]
        # positive_index_in_proba = np.argwhere(np.array(y_train) == 1).flatten()
        # y_proba_train = y_proba_train[positive_index_in_proba]
        #
        # best_alpha,best_l1_ratio,best_model,best_rmse = Build_GridSearch(model_name, params, X_train, y_proba_train, y_train) # y_train for selecting only positive samples
        # print('best model hyperparameters:',best_alpha, best_l1_ratio)
        # print('best model hyperparameters----best_rmse:', best_rmse)
        # stu_model = best_model

        # training classifier on all train data
        stu_model.fit(X_train, y_proba_train)
        print('stu_model.coef_',stu_model.coef_)

        # # save the stu_model
        # with open(file_path + f'{save_label}_simplified.pkl', 'wb') as fid:
        #     pkl.dump(stu_model, fid)

        # with open(file_path+'RI_simplified.pkl', 'rb') as file:
        #     dataee = pickle.load(file)
        # print(dataee)

    elif distill_method == "train_from_scratch":
        if distill_model == "tree":
            stu_model = DecisionTreeClassifier(max_depth=4)
        elif distill_model == "lr":
            stu_model = sklearn.pipeline.Pipeline([
                ('scale', preprocessing.StandardScaler()),
                ('model', LogisticRegression(class_weight='balanced', penalty='elasticnet', solver='saga', l1_ratio=0.5))
            ])
        else:
            raise NotImplementedError
        stu_model.fit(X_train, y_train)
        # save the stu_model
        with open(file_path + f'{save_label}_simplified.pkl', 'wb') as fid:
            pkl.dump(stu_model, fid)
    
    else:
        raise NotImplementedError

    print("For Student Model:")
    y_proba_test_stu = stu_model.predict(X_test) #

    stu_model_rmse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=False)
    stu_model_mse = mean_squared_error(y_proba_test, y_proba_test_stu, squared=True)
    print('stu_model_rmse',stu_model_rmse)
    print('stu_model_mse',stu_model_mse)

    # only focus on positive sample's rmse and mse
    X_pos_test = X_test.loc[y_test[y_test == 1].index]
    y_pos_proba_test_stu = stu_model.predict(X_pos_test)
    stu_pos_model_rmse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=False)
    stu_pos_model_mse = mean_squared_error(y_pos_proba_test, y_pos_proba_test_stu, squared=True)
    print('stu_pos_model_rmse', stu_pos_model_rmse)
    print('stu_pos_model_mse', stu_pos_model_mse)

    # compute true positive
    RI_threshold = 0.5 # 0.65
    rec_pos = np.count_nonzero(y_pos_proba_test_stu >= RI_threshold)/len(y_pos_proba_test_stu)
    print('rec_pos',rec_pos)
    y_pred_test_stu = y_proba_test_stu.copy()
    y_pred_test_stu[y_pred_test_stu>=RI_threshold] = 1
    y_pred_test_stu[y_pred_test_stu<RI_threshold] = 0

    precision_RI_stu = precision_score(y_test, y_pred_test_stu)
    print('precision_RI_stu:', precision_RI_stu)
    recall_RI_stu = recall_score(y_test, y_pred_test_stu)
    print('recall_RI_stu:', recall_RI_stu)
    confusion_matrix_RI_stu = confusion_matrix(y_test, y_pred_test_stu)
    print('confusion_matrix_stu:', confusion_matrix_RI_stu)

    aucroc = roc_auc_score(y_test, y_proba_test_stu)
    print('aucroc', aucroc)

    # precision-recall auc
    precision, recall, _ = precision_recall_curve(y_test, y_proba_test_stu)
    aucpr = auc(recall, precision)
    print('aucpr', aucpr)


    # # plot precision recall curve
    fig, ax = plt.subplots()
    ax.plot(recall, precision, color='purple')
    
    #add axis labels to plot
    ax.set_title(f'{save_label} Precision-Recall Curve')
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')
    plt.ylim(bottom=0)
    #display plot
    # plt.show()
    current_time = datetime.now()
    fig_name_0 = f"{save_label}_Precision_Recall_curve"+"_"+str(current_time)+".png"
    fig.savefig(file_path + fig_name_0)
    plt.close('all')

#  plot feature importance
    X_data = X_train
    feature_cols = feature_cols

    fig, ax = plt.subplots(figsize=(15, 12))
    fig.subplots_adjust(left=0.2, right=0.95, top=0.95, bottom=0.08, hspace=.5, wspace=.15)

    if distill_method == "train_from_scratch":
        importances = stu_model.named_steps['model'].coef_
    else:
        importances = stu_model.coef_
    
    if len(importances) == 1:
        importances = importances.reshape(-1)

    # print('importances',importances)
    num_features = len(importances)
    indices = np.argsort(abs(importances))[::-1] #
    plt.title("Feature importance")
    plt.barh(list(range(X_data.shape[1]))[:num_features], list(importances[indices])[:num_features], align="center")
    plt.yticks(list(range(X_data.shape[1]))[:num_features], (np.array(feature_cols)[indices]).tolist()[:num_features])

    # # plt.show()
    # fig_name = file_path+classifier_flag+'_student_'+'.png'
    # fig.savefig(fig_name, facecolor='white', transparent=False)
    # plt.close('all')

    return test_result

In [ ]:
C_1_result = build_classifier(X_train, y_train, X_test, y_test, feature_cols, 'classifier_1', n_fold, file_path, 1)

In [ ]:
C_3_result = build_classifier(X_train, y_train, X_test, y_test, feature_cols, 'classifier_3', n_fold, file_path, 3)